# Bibliotecas e conexão com drive

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE

In [8]:
# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Carregar o dataset
dataset_path = '/content/drive/MyDrive/ponderadas8/Cópia de bot_detection_data.csv'

# Carregar o dataset
df = pd.read_csv(dataset_path)

# Inspecionar os dados
print(df.head())

   User ID        Username                                              Tweet  \
0   132131           flong  Station activity person against natural majori...   
1   289683  hinesstephanie  Authority research natural life material staff...   
2   779715      roberttran  Manage whose quickly especially foot none to g...   
3   696168          pmason  Just cover eight opportunity strong policy which.   
4   704441          noah87                      Animal sign six data good or.   

   Retweet Count  Mention Count  Follower Count  Verified  Bot Label  \
0             85              1            2353     False          1   
1             55              5            9617      True          0   
2              6              2            4363      True          0   
3             54              5            2242      True          1   
4             26              3            8438     False          1   

       Location           Created At            Hashtags  
0     Adkinston  2020

# **Documentação + Código**

# 1. Distribuição das Classes
Antes de treinar o modelo, a distribuição das classes foi verificada para identificar possíveis desbalanceamentos entre bots e humanos:


```
print(df['Bot Label'].value_counts())

```

# 2. Tratamento de Colunas Não Numéricas
As colunas categóricas foram convertidas para valores numéricos utilizando LabelEncoder para que o modelo pudesse processá-las. Isso foi necessário pois o algoritmo RandomForest não lida diretamente com variáveis categóricas.

```
from sklearn.preprocessing import LabelEncoder

# Tratar colunas categóricas
categorical_columns = df.select_dtypes(include=['object']).columns
label_encoders = {}
for column in categorical_columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le


```
# 3. Separação de Features e Labels
O conjunto de dados foi dividido em X (features) e y (alvo). X contém todas as variáveis explicativas, enquanto y contém a variável alvo que classifica os bots e humanos.

```
X = df.drop(columns=['Bot Label'])
y = df['Bot Label']



```

# 4. Padronização dos Dados
As features foram padronizadas utilizando StandardScaler para garantir que todas as variáveis tivessem a mesma escala.

```
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


```

# 5. Balanceamento de Classes com SMOTE
Como pode haver desbalanceamento entre bots e humanos, utilizei o SMOTE (Synthetic Minority Over-sampling Technique) para balancear as classes.

```
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)



```

# 6. Divisão dos Dados em Treino e Teste
O conjunto de dados foi dividido em 80% para treino e 20% para teste, com uma semente de aleatoriedade (random_state) para garantir a reprodutibilidade dos resultados.

```
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)




```

# 7. Ajuste de Hiperparâmetros com GridSearchCV
O GridSearchCV foi utilizado para encontrar os melhores hiperparâmetros para o modelo RandomForest. Por conta do tamanho do DF foi preciso acelerar o processo utilizando 50% dos dados de treino. Os hiperparâmetros otimizados incluíram o número de estimadores (n_estimators), a profundidade máxima das árvores (max_depth), e parâmetros de amostragem.

```
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100],  # Reduzido o número de estimadores
    'max_depth': [10],  # Fixado a profundidade
    'min_samples_split': [2],  # Valor mínimo
    'min_samples_leaf': [1],  # Valor mínimo
    'bootstrap': [True]  # Mantendo o bootstrap como True
}

rf = RandomForestClassifier(random_state=42)

# Usar apenas 50% dos dados para ajuste de hiperparâmetros
X_train_subsample, _, y_train_subsample, _ = train_test_split(X_train, y_train, test_size=0.5, random_state=42)

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train_subsample, y_train_subsample)





```

# 8. Modelo Treinado
O melhor modelo foi selecionado após o ajuste dos hiperparâmetros.

```

best_rf_model = grid_search.best_estimator_


```

# 9. Avaliação do Modelo
O desempenho do modelo foi avaliado utilizando acurácia e outras métricas como precisão, recall e F1-score. A acurácia foi obtida nos dados de teste:

```

from sklearn.metrics import accuracy_score, classification_report

y_pred = best_rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia: {accuracy}')
print(classification_report(y_test, y_pred))



```


In [12]:

print(df['Bot Label'].value_counts())


print(df.select_dtypes(include=['object']).head())

from sklearn.preprocessing import LabelEncoder


categorical_columns = df.select_dtypes(include=['object']).columns
label_encoders = {}
for column in categorical_columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

X = df.drop(columns=['Bot Label'])
y = df['Bot Label']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [10],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    'bootstrap': [True]
}

rf = RandomForestClassifier(random_state=42)

X_train_subsample, _, y_train_subsample, _ = train_test_split(X_train, y_train, test_size=0.5, random_state=42)

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train_subsample, y_train_subsample)

best_rf_model = grid_search.best_estimator_

y_pred = best_rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia: {accuracy}')
print(classification_report(y_test, y_pred))

print(f'Melhores hiperparâmetros: {grid_search.best_params_}')


Bot Label
1    25018
0    24982
Name: count, dtype: int64
Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4]
Fitting 3 folds for each of 2 candidates, totalling 6 fits
Acurácia: 0.49460431654676257
              precision    recall  f1-score   support

           0       0.49      0.42      0.45      4931
           1       0.50      0.57      0.53      5077

    accuracy                           0.49     10008
   macro avg       0.49      0.49      0.49     10008
weighted avg       0.49      0.49      0.49     10008

Melhores hiperparâmetros: {'bootstrap': True, 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
